In [54]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import f_oneway
import openpyxl
import os

In [ ]:
name_sheet = '03122024'

In [190]:
data = pd.read_excel("Ren_nghe.xlsx", sheet_name=name_sheet)

Chỉ tiêu đầy đủ

In [191]:
chi_tieu = ['Chiều cao cây', 'Số lá thật', 'Chu vi gốc', 'SPAD lá già', 'SPAD lá non',
            'Số cụm hoa', 'Số nụ hoa', 'Số quả non']

Chỉ tiêu không đầy đủ cho những ngày đầu

In [157]:
chi_tieu = ['Chiều cao cây', 'Số lá thật', 'Chu vi gốc', 'SPAD lá già', 'SPAD lá non']

Chia đôi chỉ tiêu để tạo 2 ảnh

In [130]:
chi_tieu_1 = chi_tieu[:4] 
chi_tieu_2 = chi_tieu[4:] 

Test số liệu cụm hoa và nụ hoa

In [ ]:
chi_tieu_hoa = ['Số cụm hoa', 'Số nụ hoa']

data = data.dropna(subset=chi_tieu_hoa)

for ct in chi_tieu_hoa:
    mean_values = data.groupby('CT')[ct].mean()
    # std_values = data.groupby('CT')[ct].std()
    
    print(f"Giá trị trung bình của {ct}:\n", mean_values)
    # print(f"Độ lệch chuẩn của \"{ct}\":\n", std_values)

Tính toán số liệu

In [ ]:
for ct in chi_tieu:
    data[ct] = pd.to_numeric(data[ct], errors='coerce')

data = data.dropna(subset=chi_tieu)

for ct in chi_tieu:
    mean_values = data.groupby('CT')[ct].mean()
    std_values = data.groupby('CT')[ct].std()
    
    print(f"Giá trị trung bình của {ct}:\n", mean_values)
    print(f"Độ lệch chuẩn của \"{ct}\":\n", std_values)

Biểu đồ Bar-chart lưu 2 ảnh

In [ ]:
fig1, axes1 = plt.subplots(nrows=len(chi_tieu_1), ncols=1, figsize=(10, 8))

for i, ct in enumerate(chi_tieu_1):
    mean_values = data.groupby('CT')[ct].mean()
    std_values = data.groupby('CT')[ct].std()
    
    axes1[i].bar(mean_values.index, mean_values, yerr=std_values, capsize=5, color='skyblue', edgecolor='black')
    axes1[i].set_title(f'{ct} theo công thức')
    axes1[i].set_xlabel('Công thức')
    axes1[i].set_ylabel(ct)
    axes1[i].grid(axis='y', linestyle='--', alpha=0.7)

plt.tight_layout()
plt.show()

fig2, axes2 = plt.subplots(nrows=len(chi_tieu_2), ncols=1, figsize=(10, 8))

for i, ct in enumerate(chi_tieu_2):
    mean_values = data.groupby('CT')[ct].mean()
    std_values = data.groupby('CT')[ct].std()
    
    axes2[i].bar(mean_values.index, mean_values, yerr=std_values, capsize=5, color='skyblue', edgecolor='black')
    axes2[i].set_title(f'{ct} theo công thức')
    axes2[i].set_xlabel('Công thức')
    axes2[i].set_ylabel(ct)
    axes2[i].grid(axis='y', linestyle='--', alpha=0.7)

plt.tight_layout()
plt.show()

Test Anova

In [ ]:
for ct in chi_tieu:
    groups = [data[data['CT'] == ct_group][ct] for ct_group in data['CT'].unique()]
    
    f_stat, p_value = f_oneway(*groups)
    
    print(f"Phân tích ANOVA cho {ct}:")
    print(f"F-statistic: {f_stat}")
    print(f"P-value: {p_value}")
    if p_value < 0.05:
        print("Kết luận: Có sự khác biệt đáng kể giữa các công thức.\n")
    else:
        print("Kết luận: Không có sự khác biệt đáng kể giữa các công thức.\n")
    print("-" * 50)

Biểu đồ Bar-chart lưu một ảnh

In [ ]:
fig, axes = plt.subplots(nrows=len(chi_tieu), ncols=1, figsize=(10, 8))

for i, ct in enumerate(chi_tieu):
    mean_values = data.groupby('CT')[ct].mean()
    std_values = data.groupby('CT')[ct].std()
    
    axes[i].bar(mean_values.index, mean_values, yerr=std_values, capsize=5, color='skyblue', edgecolor='black')
    axes[i].set_title(f'{ct} theo công thức')
    axes[i].set_xlabel('Công thức')
    axes[i].set_ylabel(ct)
    axes[i].grid(axis='y', linestyle='--', alpha=0.7)

plt.tight_layout()
plt.show()

Số liệu Mean, Std, ANOVA lưu vào file execl

In [188]:
file_name = f'{name_sheet}.xlsx'
results = pd.DataFrame()

for ct in chi_tieu:
    mean_values = data.groupby('CT')[ct].mean()
    std_values = data.groupby('CT')[ct].std()
    
    results[f'{ct} (Mean)'] = mean_values.round(2)
    results[f'{ct} (Std)'] = std_values.round(2)
    
    groups = [data[data['CT'] == ct_group][ct] for ct_group in data['CT'].unique()]
    if len(groups) > 1:  
        anova_result = f_oneway(*groups)
        results[f'{ct} (ANOVA p-value)'] = anova_result.pvalue
        results[f'{ct} (ANOVA F)'] = anova_result.statistic

with pd.ExcelWriter(file_name, engine='openpyxl') as writer:
    columns_to_export = results.columns.tolist()
    
    results[columns_to_export].to_excel(writer, index=True, sheet_name='Kết quả')
    
    worksheet = writer.sheets['Kết quả']
    
    for col in worksheet.columns:
        max_length = max(len(str(cell.value)) for cell in col)
        worksheet.column_dimensions[col[0].column_letter].width = max_length + 2
    
    for cell in worksheet[1]:
        cell.fill = openpyxl.styles.PatternFill(start_color="FFFF00", end_color="FFFF00", fill_type="solid")

Nối các file excel

In [195]:
file_paths = ['08102024.xlsx', '15102024.xlsx', '22102024.xlsx', '29102024.xlsx',
              '05112024.xlsx', '12112024.xlsx', '19112024.xlsx', '26112024.xlsx',
              '03122024.xlsx'] 

dfs = []

for file in file_paths:
    df = pd.read_excel(file)
    dfs.append(df)

combined_df = pd.concat(dfs, ignore_index=True)

combined_df.to_excel('TongHop.xlsx', index=False)